In [9]:
import pandas as pd
import os
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import DiffbotGraphTransformer
from langchain_experimental.graph_transformers.diffbot import TypeOption
from langchain.docstore.document import Document
! pip install neo4j

In [4]:
# Access the API key
openai_api_key = 
NEO4J_URI = 
NEO4J_USERNAME = 
NEO4J_PASSWORD = 
NEO4J_DATABASE = 
DIFFBOT_API_KEY = 

# Connect to the knowledge graph instance using LangChain
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [5]:
df = pd.read_csv("JanFeb2020FilteredFixedReady.tsv", sep='\t')

# Display the DataFrame
df

,Unnamed: 0.1,Unnamed: 0,filename,compilation_vol,corpus,timeseq_not_before,era,doctype,docclass,author,compilation_name,title,doc_content,merged_content,translation
0,4200,19,人民日報_20200101_4_1,4,2020,2020-01-01,202001,NaN,1,NaN,人民日報,脱贫攻坚 万众一心加油干\n,\n\n 编者按：又是新年。时光为尺，刻写下中国人民闯关夺隘的非凡历程。\n 2019年...,脱贫攻坚 万众一心加油干\n\n\n\n 编者按：又是新年。时光为尺，刻写下中国人民闯关夺...,Poverty Alleviation: United Efforts to Forge A...
1,4204,23,人民日報_20200101_8_1,8,2020,2020-01-01,202001,NaN,1,NaN,人民日報,丹青书盛世 妙笔颂华章\n,\n\n 日月其迈，岁律更新。脱贫攻坚，步履坚定。\n 无数美术工作者踏上奋进征程，以翰...,丹青书盛世 妙笔颂华章\n\n\n\n 日月其迈，岁律更新。脱贫攻坚，步履坚定。\n 无...,"Danqing Celebrates the Prosperous Era, Masterf..."
2,4201,20,人民日報_20200101_5_1,5,2020,2020-01-01,202001,NaN,1,NaN,人民日報,中国高铁强劲奔跑\n,3.5万公里\n\n 中国高铁送上了多么令人惊喜的2019！\n 这是高铁建设成果丰硕的...,中国高铁强劲奔跑\n\n3.5万公里\n\n 中国高铁送上了多么令人惊喜的2019！\n ...,China's High-Speed Rail is Running Strong\n\n3...
3,4199,18,人民日報_20200101_3_1,3,2020,2020-01-01,202001,NaN,1,NaN,人民日報,决胜全面小康 迈向新的征程（社论）\n,\n——元旦献词\n 大江流日夜，慷慨歌未央。新中国在凯歌行进中走过70年，迈入全面建成小...,决胜全面小康 迈向新的征程（社论）\n\n\n——元旦献词\n 大江流日夜，慷慨歌未央。新...,Decisively Achieving a Moderately Prosperous S...
4,4198,17,人民日報_20200101_3_3,3,2020,2020-01-01,202001,NaN,3,NaN,人民日報,历史性成就的深刻启示（人民论坛）\n,\n\n 2019年是一个历史性年份，给人们带来了怎样的深层次思考？透过刚刚落下帷幕的“伟...,历史性成就的深刻启示（人民论坛）\n\n\n\n 2019年是一个历史性年份，给人们带来了...,Profound Insights from Historic Achievements (...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4248,3390,4441,人民日報_20200229_4_1,4,2020,2020-02-29,202002,NaN,1,NaN,人民日報,全力做好稳外贸稳外资和生活必需品保供工作\n,胡春华强调\n\n 新华社北京2月28日电 全国商务系统应对新冠肺炎疫情做好商务工作电视电...,全力做好稳外贸稳外资和生活必需品保供工作\n\n胡春华强调\n\n 新华社北京2月28日电...,"Make Every Effort to Stabilize Foreign Trade, ..."
4249,3391,4442,人民日報_20200229_5_1,5,2020,2020-02-29,202002,NaN,1,NaN,人民日報,中华人民共和国2019年国民经济和社会发展统计公报［1］\n,\n\n 2019年，面对国内外风险挑战明显上升的复杂局面，在以习近平同志为核心的党中央坚...,中华人民共和国2019年国民经济和社会发展统计公报［1］\n\n\n\n 2019年，面对...,Statistical Bulletin on National Economic and ...
4250,3392,4443,人民日報_20200229_6_1,6,2020,2020-02-29,202002,NaN,1,NaN,人民日報,附：中华人民共和国2019年国民经济和社会发展统计公报图表\n,\n\n\n,附：中华人民共和国2019年国民经济和社会发展统计公报图表\n\n\n\n\n,Attached: Statistical Bulletin of the National...
4251,3386,4437,人民日報_20200229_4_7,4,2020,2020-02-29,202002,NaN,7,NaN,人民日報,坚守在重症病房的日日夜夜\n,4名医护人员讲述——\n\n 在抗击新冠肺炎疫情的战场上，重症病房是火力最密集的主阵地。重...,坚守在重症病房的日日夜夜\n\n4名医护人员讲述——\n\n 在抗击新冠肺炎疫情的战场上，...,Staying Firm in the Intensive Care Unit Day an...


In [57]:

#from langchain_experimental.graph_transformers.diffbot import TypeOption
diffbot_transformer = DiffbotGraphTransformer(diffbot_api_key=DIFFBOT_API_KEY, fact_confidence_threshold=0.0, include_confidence=True, simplified_schema=True, extract_types=[TypeOption.FACTS])



def convert_to_graph_documents(text):
    doc = [Document(page_content=text)]
    return diffbot_transformer.convert_to_graph_documents(doc)


In [67]:
#import langchain_experimental.graph_transformers.diffbot.SimplifiedSchema
# Apply the conversion function to each entry in the 'translation_en' column
df['graph_documents'] = df['translation'].progress_apply(convert_to_graph_documents)

Could not connect to 127.0.0.1: 54379
Traceback (most recent call last):
  File "/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev/_pydevd_bundle/pydevd_comm.py", line 465, in start_client
    s.connect((host, port))
ConnectionRefusedError: [Errno 61] Connection refused
Traceback (most recent call last):
  File "/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug/pydev_jupyter_utils.py", line 81, in attach_to_debugger
    debugger.connect(pydev_localhost.get_localhost(), debugger_port)
  File "/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev/pydevd.py", line 670, in connect
    s = start_client(host, port)
  File "/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev/_pydevd_bundle/pydevd_comm.py", line 465, in start_client
    s.connect((host, port))
ConnectionRefusedError: [Errno 61] Connection refused
Failed to connect to target debugger.


In [53]:
df

,Unnamed: 0,filename,compilation_vol,corpus,timeseq_not_before,era,doctype,docclass,author,compilation_name,title,doc_content,merged_content,translation,graph_documents,has_duplicate_nodes,node_ids
0,0,人民日報_20200101_1_1,1,2020,2020-01-01,202001,NaN,1,NaN,人民日報,新年戏曲晚会在京举行\n,\n习近平李克强栗战书汪洋王沪宁赵乐际韩正王岐山出席观看\n 新华社北京12月31日电 （...,新年戏曲晚会在京举行\n\n\n习近平李克强栗战书汪洋王沪宁赵乐际韩正王岐山出席观看\n ...,New Year Opera Gala Held in Beijing\n\nXi Jinp...,[nodes=[Node(id='http://www.wikidata.org/entit...,False,"[http://www.wikidata.org/entity/Q734914, http:..."
1,1,人民日報_20200101_1_2,1,2020,2020-01-01,202001,NaN,2,NaN,人民日報,国家主席习近平发表二〇二〇年新年贺词\n,\n\n 2019年，我们用汗水浇灌收获，以实干笃定前行。成就凝结着新时代奋斗者的心血和汗...,国家主席习近平发表二〇二〇年新年贺词\n\n\n\n 2019年，我们用汗水浇灌收获，以实...,all impoverished counties on schedule.\n\nA fe...,[nodes=[] relationships=[] source=Document(pag...,False,[]
2,2,人民日報_20200101_1_3,1,2020,2020-01-01,202001,NaN,3,NaN,人民日報,中俄两国元首互致新年贺电\n,\n中俄两国总理互致新年贺电\n 新华社北京12月31日电 2019年12月31日，国家...,中俄两国元首互致新年贺电\n\n\n中俄两国总理互致新年贺电\n 新华社北京12月31日电...,New Year Greetings Exchanged Between the Leade...,[nodes=[Node(id='http://www.wikidata.org/entit...,False,"[http://www.wikidata.org/entity/Q18111, http:/..."
3,3,人民日報_20200101_1_4,1,2020,2020-01-01,202001,NaN,4,NaN,人民日報,全国政协举行新年茶话会\n,\n习近平发表重要讲话李克强栗战书王沪宁赵乐际韩正王岐山出席 汪洋主持\n 本报北京12月...,全国政协举行新年茶话会\n\n\n习近平发表重要讲话李克强栗战书王沪宁赵乐际韩正王岐山出席 ...,National Committee of the Chinese People's Pol...,[nodes=[Node(id='http://www.wikidata.org/entit...,True,"[http://www.wikidata.org/entity/Q734914, http:..."
4,4,人民日報_20200101_2_6,2,2020,2020-01-01,202001,NaN,6,NaN,人民日報,温暖人心 振奋人心\n,习近平主席新年贺词引发海外华侨华人热烈反响——\n\n “我们要万众一心加油干，越是艰险越...,温暖人心 振奋人心\n\n习近平主席新年贺词引发海外华侨华人热烈反响——\n\n “我们要...,Warmth and Inspiration\n\nPresident Xi Jinping...,"[nodes=[Node(id='Yin Chuping', type='Person', ...",True,"[Yin Chuping, Zhang Weidong, Brazil China Peac..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,488,人民日報_20200107_19_1,19,2020,2020-01-07,202001,NaN,1,NaN,人民日報,斩断“名贵特产”背后利益链（深度关注）\n,各地重拳整治利用特殊资源谋私——\n\n “全国共查处利用名贵特产、特殊资源谋取私利问题2...,斩断“名贵特产”背后利益链（深度关注）\n\n各地重拳整治利用特殊资源谋私——\n\n “...,"Cutting Off the Interest Chain Behind ""Preciou...",[nodes=[] relationships=[] source=Document(pag...,False,[]
489,489,人民日報_20200107_20_2,20,2020,2020-01-07,202001,NaN,2,NaN,人民日報,以文为鉴 可观天下\n,\n\n “盖文章，经国之大业，不朽之盛事。”\n 中国是文章大国，中国散文更是源远流长...,以文为鉴 可观天下\n\n\n\n “盖文章，经国之大业，不朽之盛事。”\n 中国是文章...,Using Literature as a Mirror to Observe the Wo...,[nodes=[] relationships=[] source=Document(pag...,False,[]
490,490,人民日報_20200107_20_3,20,2020,2020-01-07,202001,NaN,3,NaN,人民日報,用通俗理论读物讲好中国之治\n,\n\n 党的十九届四中全会明确提出坚持和完善中国特色社会主义制度、推进国家治理体系和治理...,用通俗理论读物讲好中国之治\n\n\n\n 党的十九届四中全会明确提出坚持和完善中国特色社...,Using Popular Theoretical Literature to Effect...,[nodes=[] relationships=[] source=Document(pag...,False,[]
491,491,人民日報_20200107_20_1,20,2020,2020-01-07,202001,NaN,1,NaN,人民日報,科学规划让城市更美好（开卷知新）\n,\n\n 我们在创造空间，空间也在改变我们\n 习近平总书记在上海考察期间指出，“城市是...,科学规划让城市更美好（开卷知新）\n\n\n\n 我们在创造空间，空间也在改变我们\n ...,Scientific Planning Makes Cities Better (Openi...,[nodes=[] relationships=[] source=Document(pag...,False,[]


In [41]:
text = """In 1970, his daughter Flor de Maria Fernandez was born in Beijing. But unfortunately, she suffered from septicemia. After then-Chinese premier Zhou Enlai became aware of her condition, he immediately directed medical experts for her treatment. The Chinese People's Liberation Army organized soldiers to rush to the hospital to donate blood for Arce's daughter.

Eventually, Maria returned safe. After she grew up, like her father, Maria contributed to enhancing China-Peru friendship.

Hearing Xi mention her father via the internet, the older daughter of Arce, Lourdes Fernandez Esquivel, was deeply moved. "My father was a simple and humble person," she said. "I am honored that the Chinese president has recognized the work he has done over the years to strengthen the friendship between the peoples of Peru and China."

"I once asked my father, what has China brought you? This was his answer: 'When I first came to this country, China was still poor, but possessed the courage and perseverance to overcome difficulties. China relied on its own efforts to achieve liberation, and will certainly be able to play a leading role in the future. Five thousand years of cultural and technological development and progress have supported China as a leading force in the next century."""

In [42]:
convert_to_graph_documents(text)

[GraphDocument(nodes=[Node(id='http://www.wikidata.org/entity/Q148', type='Location', properties={'name': "People's Republic of China", 'confidence': 0.999778}), Node(id='http://www.wikidata.org/entity/Q419', type='Location', properties={'name': 'Peru', 'confidence': 0.9991265}), Node(id='http://www.wikidata.org/entity/Q200106', type='Organization', properties={'name': 'P.L.A.', 'confidence': 0.99959487}), Node(id='http://www.wikidata.org/entity/Q17410', type='Person', properties={'name': 'Zhou Enlai', 'confidence': 0.9999567}), Node(id='http://www.wikidata.org/entity/Q956', type='Location', properties={'name': 'Beijing', 'confidence': 0.99757403}), Node(id='http://www.wikidata.org/entity/Q183134', type='Disease', properties={'name': 'sepsis', 'confidence': 0.822944}), Node(id='http://www.wikidata.org/entity/Q16917', type='Location', properties={'name': 'hospital', 'confidence': 0.59970176}), Node(id='http://www.wikidata.org/entity/Q4991371', type='Job title', properties={'name': 'sold

In [45]:
doc = convert_to_graph_documents(text)

In [56]:
for relationship in doc[0].relationships: 
    #print(relationship.source.id, relationship.type, relationship.target.id)
    
    relationship.properties['source'] = 'doc1'
    print(relationship.properties)

{'evidence': 'Hearing Xi mention her father via the internet, the older daughter of Arce, Lourdes Fernandez Esquivel, was deeply moved.', 'confidence': 0.92831266, 'source': 'doc1'}
{'evidence': 'Hearing Xi mention her father via the internet, the older daughter of Arce, Lourdes Fernandez Esquivel, was deeply moved.', 'confidence': 0.92831266, 'source': 'doc1'}
{'evidence': 'Hearing Xi mention her father via the internet, the older daughter of Arce, Lourdes Fernandez Esquivel, was deeply moved.', 'confidence': 0.5367857, 'source': 'doc1'}
{'evidence': 'Hearing Xi mention her father via the internet, the older daughter of Arce, Lourdes Fernandez Esquivel, was deeply moved.', 'confidence': 0.92831266, 'source': 'doc1'}
{'evidence': 'Hearing Xi mention her father via the internet, the older daughter of Arce, Lourdes Fernandez Esquivel, was deeply moved.', 'confidence': 0.92831266, 'source': 'doc1'}
{'evidence': 'Hearing Xi mention her father via the internet, the older daughter of Arce, L

[GraphDocument(nodes=[Node(id='Lourdes Fernandez Esquivel', type='Person', properties={'name': 'Lourdes Fernandez Esquivel'}), Node(id='Flor de Maria Fernandez', type='Person', properties={'name': 'Flor de Maria Fernandez', 'dateOfBirth': '1970'}), Node(id='http://www.wikidata.org/entity/Q17410', type='Person', properties={'name': 'Zhou Enlai'}), Node(id='http://www.wikidata.org/entity/Q956', type='Location', properties={'name': 'Beijing'})], relationships=[Relationship(source=Node(id='Lourdes Fernandez Esquivel', type='Person', properties={}), target=Node(id='Flor de Maria Fernandez', type='Person', properties={}), type='FAMILY_MEMBER', properties={'evidence': 'Hearing Xi mention her father via the internet, the older daughter of Arce, Lourdes Fernandez Esquivel, was deeply moved.', 'confidence': 0.92831266, 'source': 'doc1'}), Relationship(source=Node(id='Lourdes Fernandez Esquivel', type='Person', properties={}), target=Node(id='Flor de Maria Fernandez', type='Person', properties={}

In [58]:
from tqdm.notebook import tqdm
import pandas as pd
import os

# Ensure tqdm works with pandas
tqdm.pandas()

# Function to process and save each batch
def process_batch(batch, output_file):
    try:
        # Apply your function
        batch['graph_documents'] = batch['translation'].progress_apply(convert_to_graph_documents)
        # Save to a parquet file
        if os.path.exists(output_file):
            # Append to existing file
            batch.to_parquet(output_file, engine='pyarrow', append=True)
        else:
            # Create new file
            batch.to_parquet(output_file, engine='pyarrow')
    except Exception as e:
        print(f"Error processing batch: {e}")
    return batch

# Parameters
batch_size = 100
output_file = "JanFeb2020FilteredFixedReadyDiffbot.parquet"

# Check for resuming
start_index = 0
if os.path.exists(output_file):
    # If the file exists, read it to determine the last processed index
    processed_data = pd.read_parquet(output_file)
    start_index = len(processed_data)

# Process data in batches
results = []
for start in tqdm(range(start_index, len(df), batch_size), desc="Processing Batches"):
    end = start + batch_size
    batch = df.iloc[start:end].copy()  # Create a copy of the batch
    processed_batch = process_batch(batch, output_file)
    results.append(processed_batch)

# Combine all processed batches in memory (optional)
final_result = pd.concat(results, ignore_index=True) if results else None


Processing Batches:   0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Error processing batch: ("Could not convert GraphDocument(nodes=[Node(id='Jiang Xiaodan', type='Person', properties={'name': 'Jiang Xiaodan', 'positionHeld': 'journalists'}), Node(id='Liu Xingu', type='Person', properties={'name': 'Liu Xingu', 'positionHeld': 'journalists'}), Node(id='Zhao Ena', type='Person', properties={'name': 'Zhao Ena', 'positionHeld': 'journalists'}), Node(id='Huang Wenxiu', type='Person', properties={'name': 'Huang Wenxiu'}), Node(id='National Excellent Communist Party', type='Organization', properties={'name': 'National Excellent Communist Party'}), Node(id='http://www.wikidata.org/entity/Q360600', type='Person', properties={'name': 'Chen Yibing'}), Node(id='http://www.wikidata.org/entity/Q211243', type='Location', properties={'name': 'Yinchuan'}), Node(id='Dai Linfeng', type='Person', properties={'name': 'Dai Linfeng', 'positionHeld': 'journalists'}), Node(id='Zhang Yunhe', type='Person', properties={'name': 'Zhang Yunhe', 'positionHeld': 'journalists'}), Node

  0%|          | 0/100 [00:00<?, ?it/s]

Error processing batch: ('Could not convert GraphDocument(nodes=[], relationships=[], source=Document(metadata={}, page_content="Celebrating New Year\'s Day with Joyful Sports\\n\\nOn the morning of January 1st, the first Farmers\' Fitness Sports Meeting was held in Haikou Town, Daguan District, Anqing City, Anhui Province, to celebrate New Year\'s Day. Farmers\' athlete representatives from 10 villages in the town, along with town officials, participated in competitions such as cabbage carrying and tug-of-war. Everyone enjoyed the New Year\'s celebration in a joyful and harmonious atmosphere.  \\nPhoto by Xiang Shunping (Image China)")) with type GraphDocument: did not recognize Python value type when inferring an Arrow data type', 'Conversion failed for column graph_documents with type object')


...

Error processing batch: ('Could not convert GraphDocument(nodes=[Node(id=\'Su Bin\', type=\'Person\', properties={\'name\': \'Su Bin\', \'positionHeld\': \'reporter\'})], relationships=[], source=Document(metadata={}, page_content=\'Guizhou\\\'s Affordable Housing Projects Benefit 3 Million Households\\n\\nOn January 9, our reporter Su Bin reported from Guiyang: Recently, the Guizhou Provincial Department of Housing and Urban-Rural Development announced that by the end of 2018, the per capita housing area for urban residents in the province had increased to 37.93 square meters, while the per capita housing area for rural residents had reached 34.99 square meters, both more than ten times that of 1979. Among these, the affordable housing projects have resolved housing issues for over 3 million households, benefiting more than 9 million people in the province, with an urban housing security coverage rate of 25.09%.\\n\\nIn 2019, Guizhou selected 10 pilot projects for the renovation of ol

  0%|          | 0/100 [00:00<?, ?it/s]

Error processing batch: ('Could not convert GraphDocument(nodes=[Node(id=\'http://www.wikidata.org/entity/Q57390\', type=\'Person\', properties={\'name\': \'Ismail Omar Guelleh\', \'positionHeld\': \'president\'}), Node(id=\'http://www.wikidata.org/entity/Q15031\', type=\'Person\', properties={\'name\': \'Xi Jinping\', \'positionHeld\': \'president\'}), Node(id=\'http://www.wikidata.org/entity/Q3604\', type=\'Location\', properties={\'name\': \'Djibouti\'}), Node(id=\'http://www.wikidata.org/entity/Q701801\', type=\'Person\', properties={\'name\': \'Wang Yi\', \'positionHeld\': \'Foreign Minister\'})], relationships=[Relationship(source=Node(id=\'http://www.wikidata.org/entity/Q57390\', type=\'Person\', properties={}), target=Node(id=\'http://www.wikidata.org/entity/Q15031\', type=\'Person\', properties={}), type=\'WORK_RELATIONSHIP\', properties={\'evidence\': \'Wang Yi stated that under the personal care and guidance of President Xi Jinping and President Guelleh, the strategic partne

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [67]:
processed_data = pd.concat(results, ignore_index=True)
processed_rows = len(processed_data)


In [68]:
for start in tqdm(range(processed_rows, len(df), batch_size), desc="Processing Remaining Batches"):
    end = start + batch_size
    batch = df.iloc[start:end].copy()  # Create a copy of the batch
    # Process the batch and append it to `results`
    batch['graph_documents'] = batch['translation'].progress_apply(convert_to_graph_documents)
    results.append(batch)


Processing Remaining Batches:   0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

In [70]:
results[-1]

,Unnamed: 0.1,Unnamed: 0,filename,compilation_vol,corpus,timeseq_not_before,era,doctype,docclass,author,compilation_name,title,doc_content,merged_content,translation,graph_documents
4200,3301,4352,人民日報_20200228_9_4,9,2020,2020-02-28,202002,NaN,4,NaN,人民日報,坚持司法为民公正司法（适势求是）\n,把制度优势更好转化为司法效能\n\n 新中国成立70多年来，我们党领导人民创造了经济快速发...,坚持司法为民公正司法（适势求是）\n\n把制度优势更好转化为司法效能\n\n 新中国成立7...,Persisting in Justice for the People and Fair ...,[nodes=[Node(id='http://www.wikidata.org/entit...
4201,3302,4353,人民日報_20200228_9_1,9,2020,2020-02-28,202002,NaN,1,NaN,人民日報,解码·显著优势\n,\n\n 编者的话\n “立善法于天下，则天下治；立善法于一国，则一国治。”党的十九届四...,解码·显著优势\n\n\n\n 编者的话\n “立善法于天下，则天下治；立善法于一国，则...,Decoding Significant Advantages\n\nEditor's No...,[nodes=[] relationships=[] source=Document(met...
4202,3304,4355,人民日報_20200228_9_2,9,2020,2020-02-28,202002,NaN,2,NaN,人民日報,不断巩固和增强法治优势（人民观察）\n,\n\n 核心阅读\n ■中国特色社会主义法治道路凝结着我国法治实践的智慧和经验，符合我...,不断巩固和增强法治优势（人民观察）\n\n\n\n 核心阅读\n ■中国特色社会主义法治...,Continuously Consolidating and Enhancing the A...,[nodes=[Node(id='http://www.wikidata.org/entit...
4203,3291,4342,人民日報_20200228_6_4,6,2020,2020-02-28,202002,NaN,4,NaN,人民日報,在战疫一线，也在医患身边\n,人民日报电子阅报栏进驻武汉方舱医院——\n\n 这两天，武汉硚口方舱医院的大厅里，在患者人...,在战疫一线，也在医患身边\n\n人民日报电子阅报栏进驻武汉方舱医院——\n\n 这两天，武...,"On the Frontline of the Epidemic, Also by the ...",[nodes=[Node(id='http://www.wikidata.org/entit...
4204,3305,4356,人民日報_20200228_10_1,10,2020,2020-02-28,202002,NaN,1,NaN,人民日報,环卫工人的责任与担当（来自疫情防控一线的报道）\n,\n\n 版式设计：张丹峰\n,环卫工人的责任与担当（来自疫情防控一线的报道）\n\n\n\n 版式设计：张丹峰\n,The Responsibilities and Commitment of Sanitat...,[nodes=[] relationships=[] source=Document(met...
4205,3307,4358,人民日報_20200228_11_4,11,2020,2020-02-28,202002,NaN,4,NaN,人民日報,战疫一线 践行初心\n,北京3762名疫情防控工作人员申请入党\n\n 本报北京2月27日电 （记者贺勇）“我志...,战疫一线 践行初心\n\n北京3762名疫情防控工作人员申请入党\n\n 本报北京2月27...,"Fighting the Epidemic at the Frontline, Uphold...","[nodes=[Node(id='Zheng Mengmeng', type='Person..."
4206,3308,4359,人民日報_20200228_11_5,11,2020,2020-02-28,202002,NaN,5,NaN,人民日報,公安交管护航，保障路网顺畅\n,\n\n 复工复产，交通运输是“先行官”，必须打通“大动脉”，畅通“微循环”。2月21日全...,公安交管护航，保障路网顺畅\n\n\n\n 复工复产，交通运输是“先行官”，必须打通“大动...,Public Security Traffic Management Ensures Smo...,[nodes=[] relationships=[] source=Document(met...
4207,3309,4360,人民日報_20200228_11_6,11,2020,2020-02-28,202002,NaN,6,NaN,人民日報,检察机关依法对赵正永、陈国强涉嫌受贿案提起公诉\n,\n\n 本报北京2月27日电 （记者彭波）全国人民代表大会原内务司法委员会副主任委员、...,检察机关依法对赵正永、陈国强涉嫌受贿案提起公诉\n\n\n\n 本报北京2月27日电 （...,Procuratorial Authorities Lawfully Prosecute Z...,"[nodes=[Node(id='Chen Guoqiang', type='Person'..."
4208,3310,4361,人民日報_20200228_11_2,11,2020,2020-02-28,202002,NaN,2,NaN,人民日報,要对得起大伙儿的信任（一线行走）\n,\n\n 越是面对困难，基层干部以群众为重的关切，就越能换来大伙儿的暖心和安心\n \n...,要对得起大伙儿的信任（一线行走）\n\n\n\n 越是面对困难，基层干部以群众为重的关切，...,To Live Up to Everyone's Trust (On the Frontli...,"[nodes=[Node(id='Zhou Fan', type='Person', pro..."
4209,3311,4362,人民日報_20200228_11_3,11,2020,2020-02-28,202002,NaN,3,NaN,人民日報,种果苗啦！（扶贫印记）\n,\n\n 建新村坐落在广西龙胜各族自治县江底乡的深山中。2015年9月，陆恒江作为“第一书...,种果苗啦！（扶贫印记）\n\n\n\n 建新村坐落在广西龙胜各族自治县江底乡的深山中。20...,Planting Fruit Seedlings! (Poverty Alleviation...,"[nodes=[Node(id='Lu Hengjiang', type='Person',..."


In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
import os

# Ensure tqdm works with pandas
tqdm.pandas()

# Function to process and save each batch
def process_batch(batch, output_file):
    try:
        # Apply the conversion function to create 'graph_documents'
        batch['graph_documents'] = batch['translation'].progress_apply(convert_to_graph_documents)
        
        # Save to a parquet file
        if os.path.exists(output_file):
            # Append to existing file
            batch.to_parquet(output_file, engine='pyarrow', append=True)
        else:
            # Create new file if not exists
            batch.to_parquet(output_file, engine='pyarrow')
    except Exception as e:
        # Print error message if something goes wrong
        print(f"Error processing batch: {e}")
    return batch

# Parameters
batch_size = 100  # Smaller batch size
output_file = "JanFeb2020FilteredFixedReadyDiffbot.parquet"

# Check for resuming and set the starting index
start_index = 0
if os.path.exists(output_file):
    # If the file exists, read it to determine the last processed index
    processed_data = pd.read_parquet(output_file)
    start_index = len(processed_data)

# Process data in batches
results = []
for start in tqdm(range(start_index, len(df), batch_size), desc="Processing Batches"):
    end = start + batch_size
    batch = df.iloc[start:end].copy()  # Create a copy of the batch
    
    # Process each batch and append to results
    processed_batch = process_batch(batch, output_file)
    results.append(processed_batch)

# Combine all processed batches into a final DataFrame (optional, depending on your needs)
final_result = pd.concat(results, ignore_index=True) if results else None

# If needed, you can do additional work with 'final_result' here


In [28]:
# Extract all node IDs from the entire DataFrame
def extract_node_ids(row):
    # Convert row to string and extract all node ids using regex
    row_str = str(row)
    return re.findall(r"Node\(id='(.*?)'", row_str)

# Apply the extraction function to the entire column
df['node_ids'] = df['graph_documents'].apply(extract_node_ids)

# Flatten the list of node IDs and count occurrences
from collections import Counter
node_id_counter = Counter([node_id for sublist in df['node_ids'] for node_id in sublist])

# Filter for node IDs that appear in more than one document
duplicate_node_ids = {node_id: count for node_id, count in node_id_counter.items() if count > 1}

# Display the duplicate node IDs
duplicate_node_ids


{'http://www.wikidata.org/entity/Q734914': 22,
 'http://www.wikidata.org/entity/Q127295': 24,
 'http://www.wikidata.org/entity/Q18111': 6,
 'http://www.wikidata.org/entity/Q15031': 149,
 'http://www.wikidata.org/entity/Q48405': 20,
 'http://www.wikidata.org/entity/Q18113': 26,
 'http://www.wikidata.org/entity/Q64543': 27,
 'http://www.wikidata.org/entity/Q391443': 6,
 'Wu Weihua': 5,
 'Central Committee of the Jiusan Society': 4,
 'Yin Chuping': 4,
 'Zhang Weidong': 2,
 'Brazil China Peaceful Reunification Promotion Association': 2,
 'http://www.wikidata.org/entity/Q668': 7,
 'http://www.wikidata.org/entity/Q715397': 4,
 'Korea-China Cultural Friendship Association': 6,
 'La Jielian': 4,
 'Cai Rongzhuang': 4,
 'Chinese Enterprises Chamber of Commerce': 6,
 'Jin Linze': 6,
 'Qu Huan': 3,
 'http://www.wikidata.org/entity/Q11636656': 6,
 'School of International Service at American University': 3,
 'http://www.wikidata.org/entity/Q869': 7,
 'http://www.wikidata.org/entity/Q13415224': 2,
 

In [72]:
kg.refresh_schema()

In [73]:
import pickle

# Pickle the 'results' list to a file
with open('results.pkl', 'wb') as f:
    pickle.dump(results, f)


In [75]:
import pandas as pd

# Step 1: Combine DataFrames
df_combined = pd.concat(results, ignore_index=True)

# Step 2: Update the graph_documents column
for index, row in df_combined.iterrows():
    graph_document = row['graph_documents'][0]  # Access the first and only GraphDocument in the list
    
    # Extract filename and date_published
    filename = row['filename']
    date_published = row['timeseq_not_before']  
    
    # Update relationships
    for relationship in graph_document.relationships:
        relationship.properties['source'] = filename
        relationship.properties['date_published'] = date_published

# Step 3: Collect all updated graph documents
updated_graph_documents = [row['graph_documents'][0] for index, row in df_combined.iterrows()]

# Step 4: Add the updated graph documents to the knowledge graph
kg.add_graph_documents(updated_graph_documents)


In [77]:
df_combined.to_csv('JanFebDiffbot.tsv', sep='\t')

In [79]:
df_combined[df_combined["filename"] == "人民日報_20200107_9_5"]

,Unnamed: 0.1,Unnamed: 0,filename,compilation_vol,corpus,timeseq_not_before,era,doctype,docclass,author,compilation_name,title,doc_content,merged_content,translation,graph_documents
420,4619,438,人民日報_20200107_9_5,9,2020,2020-01-07,202001,NaN,5,NaN,人民日報,英勇制敌 舍己为人（为了民族复兴·英雄烈士谱）\n,彭宝林：\n\n 据新华社电 （记者李雄鹰）彭宝林，生于1966年8月，广东惠阳县马安镇...,英勇制敌 舍己为人（为了民族复兴·英雄烈士谱）\n\n彭宝林：\n\n 据新华社电 （记...,"Heroically Defeating the Enemy, Selflessly Ser...","[nodes=[Node(id='Weng Zhenming', type='Person'..."
